# PilotNet LIF Benchmarking Example

This tutorial demonstrates how to use __lava__ to perform __energy__, __execution__, __memory__ and __activity__ on a PilotNet LIF network running on Loihi 2. _Note: This tutorial only runs on Lohi 2 backend._

# TODO change graphics
![PilotNet Inference](images/pilotnet_lif.PNG)

Refer to [run.ipynb](https://github.com/lava-nc/lava-dl/blob/main/tutorials/lava/lib/dl/netx/pilotnet_snn/run.ipynb) for detailed walkthrough of how to inference the network on both CPU and Loihi 2.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from lava.magma.core.run_configs import Loihi2SimCfg, Loihi2HwCfg
from lava.magma.core.run_conditions import RunSteps
# from lava.proc import io
# from lava.magma.core.process.variable import Var
# from lava.magma.core.process.ports.ports import RefPort

from lava.lib.dl import netx
from dataset import PilotNetDataset
# from utils import (
#     PilotNetEncoder, PilotNetDecoder, VoltageReader, PilotNetMonitor,
#     loihi2hw_exception_map, loihi2sim_exception_map
# )


# Import modules for Loihi2 execution

Check if Loihi2 compiker is available and import related modules.

In [ ]:
from lava.utils.system import Loihi2
Loihi2.preferred_partition = 'kp_build'
loihi2_is_available = Loihi2.is_loihi2_available

if loihi2_is_available:
    print(f'Running on {Loihi2.partition}')
    from lava.utils import loihi2_profiler
else:
    RuntimeError("Loihi2 compiler is not available in this system. "
                 "This tutorial cannot proceed further.")

## Create network block

PilotNet LIF is described by the hdf5 file inference `network.net`.

In [ ]:
net = netx.hdf5.Network(net_config='network.net',
                        reset_interval=16,
                        reset_offset=3)
print(net)


## Set execution parameters
Configure number of samples, execution timesteps, and readout offset.

In [ ]:
steps_per_sample = net.reset_interval
num_steps = 10000  # Run for a very long time to get good power measurement


## Create Dataset instance
Typically the user would write it or provide it.

In [ ]:
full_set = PilotNetDataset(
    path='../data',
    transform=net.in_layer.transform,  # input transform
    visualize=True,  # visualize ensures the images are returned in sequence
    sample_offset=10550,
)

## Write a representative frame to the network
Here, we will configure the network to repeatedly run on a single frame to avoid IO bottleneck.

In [ ]:
frame_id = 100
image, gt = full_set[frame_id]
net.in_layer.neuron.bias = image


## Configure profiling tools

In [ ]:
power_logger = loihi2_profiler.Loihi2Power(num_steps=num_steps)
runtime_logger = loihi2_profiler.Loihi2ExecutionTime()
memory_logger = loihi2_profiler.Loihi2Memory()
activity_logger = loihi2_profiler.Loihi2Activity()

pre_run_fxs = [
    lambda b: power_logger.attach(b),
    lambda b: runtime_logger.attach(b),
    lambda b: memory_logger.attach(b),
    lambda b: activity_logger.attach(b),
]
post_run_fxs = [
    lambda b: power_logger.get_results(),
    lambda b: runtime_logger.get_results(),
    lambda b: memory_logger.get_results(),
    lambda b: activity_logger.get_results(),
]

## Run the network

In [ ]:
run_config = Loihi2HwCfg()
net.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_config)
net.stop()

## Evaluate Results
Plot and compare the results with the dataset ground truth.

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(np.array(gts), label='Ground Truth')
plt.plot(result[1:].flatten(), label='Lava output')
plt.xlabel(f'Sample frames (+10550)')
plt.ylabel('Steering angle (radians)')
plt.legend()
